<h1>RecomendAI</h1>


---



Sistema de recomendação de filmes.

# RecomendAI — Resumo Executivo

**Objetivo.** Recomendar filmes (Top-N) a cada usuário com base no seu **histórico de avaliações**.

**Como funciona.**
- Aprendemos **representações (embeddings)** para usuários e filmes com uma rede neural.
- Calculamos uma **pontuação de afinidade** (via produto interno entre embeddings) para cada filme que o usuário **ainda não viu**.
- Ordenamos por essa pontuação e retornamos o **Top-N** como recomendações.

**Dados (MovieLens 100k).**
- 100.000 avaliações de 943 usuários em 1.682 filmes (`u.data`, `u.item`).
- Escolhido por ser leve, clássico e amplamente utilizado em pesquisa/ensino.

**Fluxo do notebook.**
1. Preparação e download do dataset.
2. Leitura e EDA (distribuições, popularidade etc.).
3. Pré-processamento (mapeamento de IDs, divisão treino/teste).
4. Modelo (embeddings de usuário/filme + produto interno).
5. Recomendação: `recomendar_para_usuario(id)` retorna o Top-N com título, gêneros, média global e **pontuação prevista**.

# Preparação do Dataset MovieLens 100k

Estas etapas são referentes aos processos de limpeza, preparação e organização dos dados.

## 1. Instalação e Importação de Dependências

Abaixo, realizamos a instalação as bibliotecas necessárias para este projeto.

- **pandas**: Para manipulação e análise de dados em formato tabular (DataFrames).
- **numpy**: Para operações numéricas e com arrays.
- **requests**: Para baixar o arquivo do dataset da internet.
- **tqdm**: Para exibir barras de progresso durante o download.

In [1]:
%pip install pandas numpy requests tqdm

## 2. Download e Extração dos Dados

Nesta etapa, faremos o download do arquivo compactado do dataset MovieLens 100k e o extrairemos em uma pasta local.

In [2]:
import os
import requests
import zipfile
from tqdm.auto import tqdm

url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
zip_path = 'ml-100k.zip'
extract_path = 'data/'

os.makedirs(extract_path, exist_ok=True)

print(f"Baixando o dataset de: {url}")
response = requests.get(url, stream=True)
total_size_in_bytes = int(response.headers.get('content-length', 0))
block_size = 1024 # 1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

with open(zip_path, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()

if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("Erro durante o download.")
else:
    print("Download concluído com sucesso.")

print(f"Extraindo o dataset para: {extract_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extraído com sucesso na pasta: {extract_path}")

Baixando o dataset de: http://files.grouplens.org/datasets/movielens/ml-100k.zip


  0%|          | 0.00/4.92M [00:00<?, ?iB/s]

Download concluído com sucesso.
Extraindo o dataset para: data/
Dataset extraído com sucesso na pasta: data/


## 3. Carregamento dos Dados

Nesta etapa, carregaremos os principais arquivos do dataset MovieLens 100k em DataFrames pandas.

- **u.data**: Contém as avaliações dos usuários (user_id, item_id, rating, timestamp).
- **u.item**: Contém informações sobre os filmes (item_id, title, release_date, video_release_date, IMDb_URL, e 19 colunas binárias para gêneros).
- **u.genre**: Contém a lista de gêneros (genre_name, genre_id).

In [3]:
import pandas as pd

data_path = 'data/ml-100k/u.data'
item_path = 'data/ml-100k/u.item'
genre_path = 'data/ml-100k/u.genre'

ratings_cols = ['user_id', 'item_id', 'rating', 'timestamp']
ratings_df = pd.read_csv(data_path, sep='\t', names=ratings_cols)

movie_cols = ['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL']
genre_cols = [
    'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
    'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
    'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]
all_movie_cols = movie_cols + genre_cols
movies_df = pd.read_csv(item_path, sep='|', names=all_movie_cols, encoding='latin-1')

genre_cols = ['genre_name', 'genre_id']
genres_df = pd.read_csv(genre_path, sep='|', names=genre_cols)

movies_ds_complete = movies_df.copy()

print("DataFrame de Avaliações (ratings_df):")
display(ratings_df.head())

print("\nDataFrame de Filmes (movies_df):")
display(movies_df.head())

print("\nDataFrame de Gêneros (genres_df):")
display(genres_df.head())

DataFrame de Avaliações (ratings_df):


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596



DataFrame de Filmes (movies_df):


,item_id,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



DataFrame de Gêneros (genres_df):


,genre_name,genre_id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


## 4. Limpeza dos Dados

Nesta etapa, faremos a limpeza inicial dos dados, removendo colunas irrelevantes e padronizando os títulos dos filmes.

In [4]:
movies_df = movies_df.drop(columns=['video_release_date', 'IMDb_URL'])

movies_df['title'] = movies_df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
movies_df['title'] = movies_df['title'].str.strip()

duplicate_movies = movies_df[movies_df.duplicated('item_id')]

if not duplicate_movies.empty:
    print("Foram encontradas duplicatas no campo 'item_id':")
    display(duplicate_movies)
else:
    print("Nenhuma duplicata encontrada no campo 'item_id'.")

print("\nDataFrame de Filmes (movies_df) após a limpeza:")
display(movies_df.head())

Nenhuma duplicata encontrada no campo 'item_id'.

DataFrame de Filmes (movies_df) após a limpeza:


,item_id,title,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye,01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty,01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat,01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


## 5. Transformação de Gêneros

Nesta etapa, vamos consolidar as informações de gênero em uma única coluna e criar um DataFrame onde cada linha representa um par filme-gênero.

In [5]:
genre_cols = [
    'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
    'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
    'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]

movies_df['genres'] = movies_df.apply(
    lambda row: [col for col in genre_cols if row[col] == 1], axis=1
)

movies_df = movies_df.drop(columns=genre_cols)

print("DataFrame de Filmes (movies_df) com a coluna 'genres':")
display(movies_df.head())

movie_genres_exploded_df = movies_df.explode('genres')

print("\nDataFrame Explodido (movie_genres_exploded_df - combinações filme-gênero):")
display(movie_genres_exploded_df.head())

DataFrame de Filmes (movies_df) com a coluna 'genres':


,item_id,title,release_date,genres
0,1,Toy Story,01-Jan-1995,"[Animation, Children's, Comedy]"
1,2,GoldenEye,01-Jan-1995,"[Action, Adventure, Thriller]"
2,3,Four Rooms,01-Jan-1995,[Thriller]
3,4,Get Shorty,01-Jan-1995,"[Action, Comedy, Drama]"
4,5,Copycat,01-Jan-1995,"[Crime, Drama, Thriller]"



DataFrame Explodido (movie_genres_exploded_df - combinações filme-gênero):


,item_id,title,release_date,genres
0,1,Toy Story,01-Jan-1995,Animation
0,1,Toy Story,01-Jan-1995,Children's
0,1,Toy Story,01-Jan-1995,Comedy
1,2,GoldenEye,01-Jan-1995,Action
1,2,GoldenEye,01-Jan-1995,Adventure


## 6. Agregação de Avaliações (Ratings)

Nesta etapa, utilizaremos o DataFrame de avaliações (`ratings_df`) para calcular métricas importantes sobre a popularidade e qualidade percebida de cada filme: a média das avaliações (`avg_rating`) e o número total de avaliações (`rating_count`). Essas métricas serão adicionadas ao DataFrame de filmes (`movies_df`).

In [6]:
movie_ratings_agg = ratings_df.groupby('item_id')['rating'].agg(['mean', 'count']).reset_index()

movie_ratings_agg.columns = ['item_id', 'avg_rating', 'rating_count']

movies_df = pd.merge(movies_df, movie_ratings_agg, on='item_id', how='left')

movies_df['avg_rating'] = movies_df['avg_rating'].fillna(0)
movies_df['rating_count'] = movies_df['rating_count'].fillna(0).astype(int)

print("DataFrame de Filmes (movies_df) com métricas de avaliação:")
display(movies_df.head())

DataFrame de Filmes (movies_df) com métricas de avaliação:


,item_id,title,release_date,genres,avg_rating,rating_count
0,1,Toy Story,01-Jan-1995,"[Animation, Children's, Comedy]",3.878319,452
1,2,GoldenEye,01-Jan-1995,"[Action, Adventure, Thriller]",3.206107,131
2,3,Four Rooms,01-Jan-1995,[Thriller],3.033333,90
3,4,Get Shorty,01-Jan-1995,"[Action, Comedy, Drama]",3.550239,209
4,5,Copycat,01-Jan-1995,"[Crime, Drama, Thriller]",3.302326,86


## 7. Armazenamento dos dados pré-processados

Nesta etapa, salvaremos os DataFrames resultantes do pré-processamento em uma pasta local. Esta é uma etapa importante para garantir que os dados limpos e estruturados estejam prontos para serem utilizados na etapa de modelagem do sistema de recomendação, sem a necessidade de repetir todo o pipeline de preparação.

In [7]:
import os

preprocessed_path = 'preprocessed'
os.makedirs(preprocessed_path, exist_ok=True)

movies_save_path_csv = os.path.join(preprocessed_path, 'movies.csv')
movies_exploded_save_path_csv = os.path.join(preprocessed_path, 'movies_exploded.csv')
ratings_save_path_csv = os.path.join(preprocessed_path, 'ratings.csv')

try:
    print(f"Salvando movies_df em: {movies_save_path_csv}")
    movies_df.to_csv(movies_save_path_csv, index=False)

    print(f"Salvando movie_genres_exploded_df em: {movies_exploded_save_path_csv}")
    movie_genres_exploded_df.to_csv(movies_exploded_save_path_csv, index=False)

    print(f"Salvando ratings_df em: {ratings_save_path_csv}")
    ratings_df.to_csv(ratings_save_path_csv, index=False)

    print("\nArquivos pré-processados salvos com sucesso em formato CSV!")
    print(f"Os arquivos se encontram na pasta: {preprocessed_path}")
    print("Estes arquivos representam a versão final dos dados prontos para serem utilizados na etapa de modelagem do sistema de recomendação.")

except Exception as e:
    print(f"Erro ao salvar arquivos CSV: {e}")
    print("Não foi possível salvar os arquivos pré-processados.")

Salvando movies_df em: preprocessed/movies.csv
Salvando movie_genres_exploded_df em: preprocessed/movies_exploded.csv
Salvando ratings_df em: preprocessed/ratings.csv

Arquivos pré-processados salvos com sucesso em formato CSV!
Os arquivos se encontram na pasta: preprocessed
Estes arquivos representam a versão final dos dados prontos para serem utilizados na etapa de modelagem do sistema de recomendação.



# Arquitetura do Modelo

### 8. Arquitetura do Modelo de Recomendação (Keras/TensorFlow)

Nesta seção, definiremos a arquitetura da rede neural para o nosso sistema de recomendação utilizando Keras e TensorFlow. O modelo será baseado em embeddings para usuários e filmes.
Para avaliar a qualidade das recomendações Top-N geradas pelo modelo, precisamos separar um conjunto de dados que simule o cenário real de recomendação. Isso envolve segurar (hold out) algumas das avaliações mais recentes ou aleatórias de cada usuário, para que o modelo não "veja" essas avaliações durante o treinamento. Depois de treinar o modelo nos dados restantes, podemos gerar recomendações para os usuários e verificar quantos dos filmes no conjunto "segurado" aparecem na lista de recomendações.

Nesta etapa, vamos dividir o `ratings_df` em dois conjuntos:

- **ratings_train_model**: Usado para treinar e validar o modelo de previsão de notas (o que já fizemos, mas faremos a divisão novamente para clareza).
- **ratings_test_recommendation**: Usado para avaliar as recomendações Top-N. Este conjunto conterá algumas avaliações "escondidas" por usuário.

In [8]:
from sklearn.model_selection import train_test_split

test_size_recommendation = 0.2

ratings_train_list = []
ratings_test_list = []

for user_id, group in ratings_df.groupby('user_id'):
    if len(group) < 2:
        ratings_train_list.append(group)
    else:
        train_group, test_group = train_test_split(
            group,
            test_size=test_size_recommendation,
            random_state=42,
        )
        ratings_train_list.append(train_group)
        ratings_test_list.append(test_group)

ratings_train_model = pd.concat(ratings_train_list).sample(frac=1, random_state=42).reset_index(drop=True)
ratings_test_recommendation = pd.concat(ratings_test_list).sample(frac=1, random_state=42).reset_index(drop=True)


print(f"Shape do ratings_df original: {ratings_df.shape}")
print(f"Shape do ratings_train_model (para treinamento do modelo): {ratings_train_model.shape}")
print(f"Shape do ratings_test_recommendation (para avaliação de recomendação): {ratings_test_recommendation.shape}")

print(f"\nNúmero de usuários únicos no ratings_df original: {ratings_df['user_id'].nunique()}")
print(f"Número de usuários únicos no ratings_train_model: {ratings_train_model['user_id'].nunique()}")
print(f"Número de usuários únicos no ratings_test_recommendation: {ratings_test_recommendation['user_id'].nunique()}")

print("\nPrimeiras linhas do ratings_train_model:")
display(ratings_train_model.head())

print("\nPrimeiras linhas do ratings_test_recommendation:")
display(ratings_test_recommendation.head())

Shape do ratings_df original: (100000, 4)
Shape do ratings_train_model (para treinamento do modelo): (79619, 4)
Shape do ratings_test_recommendation (para avaliação de recomendação): (20381, 4)

Número de usuários únicos no ratings_df original: 943
Número de usuários únicos no ratings_train_model: 943
Número de usuários únicos no ratings_test_recommendation: 943

Primeiras linhas do ratings_train_model:


,user_id,item_id,rating,timestamp
0,365,1137,5,891303950
1,67,117,5,875379794
2,721,1119,4,877147795
3,497,394,3,878759862
4,655,751,3,888474960



Primeiras linhas do ratings_test_recommendation:


,user_id,item_id,rating,timestamp
0,385,92,3,879443217
1,216,238,5,880244446
2,65,216,4,879217912
3,551,708,1,892783830
4,648,769,1,884883724


### 8.1. Definição e Treinamento do Modelo de Embeddings

Nesta seção, definimos a arquitetura do modelo de recomendação utilizando Keras/TensorFlow. O modelo utiliza camadas de embedding para usuários e filmes, calcula o produto interno entre os embeddings para prever a avaliação e é compilado com um otimizador Adam e função de perda MSE. Em seguida, o modelo é treinado utilizando os dados de treinamento (`ratings_train_model`), com validação em um subconjunto desses dados.

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

num_users = ratings_train_model['user_id'].nunique()
num_movies = ratings_train_model['item_id'].nunique()

embedding_size = 50

user_id_input = Input(shape=(1,), name='user_id')
movie_id_input = Input(shape=(1,), name='movie_id')

max_user_id = ratings_df['user_id'].max()
max_item_id = ratings_df['item_id'].max()

user_embedding = Embedding(input_dim=max_user_id + 1, output_dim=embedding_size, name='user_embedding')(user_id_input)
movie_embedding = Embedding(input_dim=max_item_id + 1, output_dim=embedding_size, name='movie_embedding')(movie_id_input)

user_vector = Flatten()(user_embedding)
movie_vector = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vector, movie_vector])

output = tf.keras.layers.Lambda(lambda x: x * 4 + 1)(tf.keras.activations.sigmoid(dot_product))


model = Model(inputs=[user_id_input, movie_id_input], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae', 'RootMeanSquaredError'])

model.summary()

user_ids_model_train = ratings_train_model['user_id']
movie_ids_model_train = ratings_train_model['item_id']
y_model_train = ratings_train_model['rating']

user_ids_train, user_ids_val, movie_ids_train, movie_ids_val, y_train, y_val = train_test_split(
    user_ids_model_train, movie_ids_model_train, y_model_train, test_size=0.2, random_state=42
)

X_train = {'user_id': user_ids_train, 'movie_id': movie_ids_train}
X_val = {'user_id': user_ids_val, 'movie_id': movie_ids_val}


epochs = 5
batch_size = 64

print("\nIniciando o treinamento do modelo com ratings_train_model...")

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    verbose=1
)

print("\nTreinamento concluído.")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_id             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_id            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 50)     │     47,200 │ user_id[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_embedding     │ (None, 1, 50)     │     84,150 │ movie_id[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ movie_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sigmoid (Sigmoid)   │ (None, 1)         │          0 │ dot[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ sigmoid[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 131,350 (513.09 KB)

 Trainable params: 131,350 (513.09 KB)

 Non-trainable params: 0 (0.00 B)


Iniciando o treinamento do modelo com ratings_train_model...
Epoch 1/5
996/996 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - RootMeanSquaredError: 1.2338 - loss: 1.5224 - mae: 0.9947 - val_RootMeanSquaredError: 1.0783 - val_loss: 1.1627 - val_mae: 0.8820
Epoch 2/5
996/996 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 1.0017 - loss: 1.0035 - mae: 0.8092 - val_RootMeanSquaredError: 0.9818 - val_loss: 0.9640 - val_mae: 0.7821
Epoch 3/5
996/996 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 0.9076 - loss: 0.8239 - mae: 0.7212 - val_RootMeanSquaredError: 0.9561 - val_loss: 0.9141 - val_mae: 0.7582
Epoch 4/5
996/996 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - RootMeanSquaredError: 0.8343 - loss: 0.6961 - mae: 0.6620 - val_RootMeanSquaredError: 0.9448 - val_loss: 0.8927 - val_mae: 0.7480
Epoch 5/5
996/996 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 0.7743 - loss: 0.5995 - mae: 0.6130 - val_RootMeanSquaredError: 0.9428 - val_loss: 0.8888 - val_mae: 0.7454

Treinamento concluído.

Após isso, faremos uma nova filtragem e ranqueamneto. Nessa nova etapa, iremos ranquear os filmes que o modelo indicou com base na similaridade entre a sinopse de um filme que o usuário já mostrou gostar e os filmes indicados. Além disso, consideraremos o inverso da similaridade entre um filme que o usuário não gostou também.

In [10]:
def get_sinopse(title, year):
    if title.endswith(", The"):
        title = "The " + title.replace(", The", "")

    params = {
        "t": title,
        "y": year,
        "apikey": API_KEY
    }

    response = requests.get(BASE_URL, params=params)

    data = response.json()
    plot = data.get("Plot", "N/A") if data.get("Response") == "True" else "Title not found"
    return plot

In [11]:
import regex as re
import requests
from sentence_transformers import SentenceTransformer, util


BASE_URL = "http://www.omdbapi.com/"
API_KEY = "ec50a218"
def get_top5(recomendacoes_para_usuario):
  recomendacoes = {'title': [], 'nota_prevista': [], 'sinopse': [], "year": []}

  for idx, row in recomendacoes_para_usuario.iterrows():
      release_date = movies_ds_complete.iloc[row['item_id'] - 1]['release_date']
      year = re.findall(r'\d{4}', release_date)[0]
      title = row['title']
      plot = get_sinopse(title, year)
      recomendacoes['year'].append(year)
      recomendacoes['title'].append(title)
      recomendacoes['nota_prevista'].append(row['nota_prevista'])
      recomendacoes['sinopse'].append(plot)

  exemplo_df = ratings_df.loc[ratings_df['user_id'] == id_de_exemplo]

  linha_max = exemplo_df.loc[exemplo_df['rating'].idxmax()]
  linha_min = exemplo_df.loc[exemplo_df['rating'].idxmin()]

  linha_min = movies_df.loc[movies_df.item_id == linha_min['item_id']]
  name = linha_min['title'].iloc[0]
  year = re.findall(r'\d{4}', linha_min['release_date'].iloc[0])[0]
  plot_min = get_sinopse(name, year)

  linha_max = movies_df.loc[movies_df.item_id == linha_max['item_id']]
  name = linha_max['title'].iloc[0]
  year = re.findall(r'\d{4}', linha_max['release_date'].iloc[0])[0]
  plot_max = get_sinopse(name, year)

  encoder = SentenceTransformer("all-MiniLM-L6-v2")

  sinopse_gostou = plot_max
  sinopse_nao_gostou = plot_min
  candidatos = recomendacoes['sinopse']

  emb_gostou = encoder.encode(sinopse_gostou, convert_to_tensor=True)
  emb_nao_gostou = encoder.encode(sinopse_nao_gostou, convert_to_tensor=True)
  emb_candidatos = encoder.encode(candidatos, convert_to_tensor=True)

  sim_positiva = util.cos_sim(emb_candidatos, emb_gostou)
  sim_negativa = util.cos_sim(emb_candidatos, emb_nao_gostou)
  scores = sim_positiva - sim_negativa

  ranking = sorted(
      zip(recomendacoes['title'], recomendacoes['sinopse'], scores),
      key=lambda x: x[2],
      reverse=True
  )

  top5 = ranking[:5]
  titulos = list()
  for titulo, sinopse, score in top5:
    titulos.append(titulo)
  return titulos


### 9. Gerando Recomendações para um Usuário Específico

Com o modelo treinado, podemos agora utilizá-lo para gerar recomendações de filmes para um usuário específico. A função `recomendar_para_usuario` recebe o ID do usuário, o modelo treinado e o número de recomendações desejadas. Ela identifica os filmes que o usuário ainda não avaliou, prevê a avaliação para esses filmes utilizando o modelo e retorna uma lista dos filmes com as maiores notas previstas, juntamente com informações relevantes como título, gêneros e a média de avaliação geral do filme.

In [12]:
def recomendar_para_usuario(id_do_usuario, modelo_treinado, num_recomendacoes=10):
    todos_os_ids_de_filmes = ratings_df['item_id'].unique()
    filmes_ja_vistos = ratings_df[ratings_df['user_id'] == id_do_usuario]['item_id'].values
    filmes_nao_vistos = np.setdiff1d(todos_os_ids_de_filmes, filmes_ja_vistos)

    input_usuario_array = np.full(len(filmes_nao_vistos), id_do_usuario)
    previsoes = modelo_treinado.predict([input_usuario_array, filmes_nao_vistos])

    resultados = pd.DataFrame({'item_id': filmes_nao_vistos, 'nota_prevista': previsoes.flatten()})
    top_recomendacoes = resultados.sort_values(by='nota_prevista', ascending=False).head(num_recomendacoes)
    recomendacoes_finais = top_recomendacoes.merge(movies_df, on='item_id')

    top_5_recomendacoes = get_top5(recomendacoes_finais)
    print(top_5_recomendacoes)

    df_filtrado = recomendacoes_finais[
        recomendacoes_finais['title'].isin(top_5_recomendacoes)
    ].copy()

    df_filtrado['title'] = pd.Categorical(
        df_filtrado['title'],
        categories=top_5_recomendacoes,
        ordered=True
    )

    df_ordenado = df_filtrado.sort_values('title')

    return df_ordenado[['item_id', 'title', 'genres', 'avg_rating', 'nota_prevista']]


### 10. Demonstração da Recomendação

Nesta seção, vamos colocar nossa função de recomendação em prática. Escolheremos um usuário de exemplo e executaremos a função `recomendar_para_usuario` para ver quais filmes o modelo sugere para ele. Os resultados serão exibidos em uma tabela, mostrando o ID do filme, título, gêneros, a média de avaliação geral do filme no dataset e a nota prevista pelo nosso modelo especificamente para o usuário escolhido.

In [13]:
import pandas as pd
import numpy as np

id_de_exemplo = 20

print(f"Buscando recomendações para o usuário de ID: {id_de_exemplo}...")
print("Isso pode levar alguns segundos, pois a IA está analisando todos os filmes que ele ainda não viu.")

recomendacoes_para_usuario = recomendar_para_usuario(id_de_exemplo, model)

print(f"\n Top 10 Recomendações para o Usuário {id_de_exemplo}:")

display(recomendacoes_para_usuario)

print("\nAvaliando o modelo no conjunto de validação...")

try:
    loss, mae, rmse = model.evaluate(X_val, y_val, verbose=0)
    print(f"Métricas no conjunto de validação: MSE={loss:.4f}, MAE={mae:.4f}, RMSE={rmse:.4f}")
except NameError:
    print("Conjuntos de validação (X_val, y_val) não encontrados. Execute a célula de treinamento primeiro.")

Buscando recomendações para o usuário de ID: 20...
Isso pode levar alguns segundos, pois a IA está analisando todos os filmes que ele ainda não viu.
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['With Honors', 'Tin Cup', 'MatchMaker, The', 'First Knight', 'Star Trek III: The Search for Spock']

 Top 10 Recomendações para o Usuário 20:


,item_id,title,genres,avg_rating,nota_prevista
9,941,With Honors,"[Comedy, Drama]",3.065217,3.595041
1,284,Tin Cup,"[Comedy, Romance]",3.160622,3.713527
4,749,"MatchMaker, The","[Comedy, Romance]",3.176471,3.647544
8,720,First Knight,"[Action, Adventure, Drama, Romance]",3.023256,3.603779
6,229,Star Trek III: The Search for Spock,"[Action, Adventure, Sci-Fi]",3.111111,3.619053



Avaliando o modelo no conjunto de validação...
Métricas no conjunto de validação: MSE=0.8888, MAE=0.7454, RMSE=0.9428




---



### 11. Avaliação das Recomendações Top-N

Nesta seção, avaliaremos a qualidade das recomendações geradas pelo nosso modelo utilizando métricas de avaliação Top-N. Usaremos o conjunto de dados `ratings_test_recommendation`, que contém as avaliações "seguradas" para este fim.

As métricas que calcularemos são:

- **Precisão@N**: A proporção de filmes recomendados que o usuário realmente avaliou no conjunto de teste.
- **Recall@N**: A proporção de filmes que o usuário avaliou no conjunto de teste que apareceram na lista de recomendações.

Calcularemos essas métricas para um valor específico de N (por exemplo, N=10, o tamanho da nossa lista de recomendações).

In [14]:
import numpy as np
import pandas as pd

N = 10

def calculate_precision_recall(recommended_items, test_items, n):
    recommended_at_n = recommended_items[:n]

    hits = len(set(recommended_at_n).intersection(set(test_items)))

    precision = hits / n if n > 0 else 0

    recall = hits / len(test_items) if len(test_items) > 0 else 0

    return precision, recall

all_precisions = []
all_recalls = []

test_users = ratings_test_recommendation['user_id'].unique()

print(f"Avaliando recomendações Top-{N} para {len(test_users)} usuários no conjunto de teste...")

for user_id in test_users:
    actual_test_movies = ratings_test_recommendation[
        ratings_test_recommendation['user_id'] == user_id
    ]['item_id'].tolist()

    try:
        recommended_df = recomendar_para_usuario(user_id, model, num_recomendacoes=N)
        recommended_movies = recommended_df['item_id'].tolist()

        precision, recall = calculate_precision_recall(recommended_movies, actual_test_movies, N)

        all_precisions.append(precision)
        all_recalls.append(recall)

    except Exception as e:
        print(f"Erro ao gerar recomendações ou calcular métricas para o usuário {user_id}: {e}")
        all_precisions.append(0)
        all_recalls.append(0)


average_precision = np.mean(all_precisions) if all_precisions else 0
average_recall = np.mean(all_recalls) if all_recalls else 0

print("\n--- Resultados da Avaliação Top-N ---")
print(f"Precisão Média @{N}: {average_precision:.4f}")
print(f"Recall Médio @{N}: {average_recall:.4f}")

print("\nInterpretação:")
print(f"- Precisão Média @{N}: Em média, {average_precision:.1%} dos filmes recomendados estão no conjunto de teste do usuário.")
print(f"- Recall Médio @{N}: Em média, {average_recall:.1%} dos filmes que o usuário avaliou no teste foram incluídos nas {N} principais recomendações.")
print("\nEstes valores fornecem uma indicação do quão bem o sistema é capaz de 'recuperar' itens relevantes para o usuário no conjunto de teste.")

Avaliando recomendações Top-10 para 943 usuários no conjunto de teste...
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
['Man Who Would Be King, The', 'Ran', 'Postino, Il', 'Treasure of the Sierra Madre, The', 'Top Hat']
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
['Boot, Das', 'Good Will Hunting', 'Godfather, The', '12 Angry Men', 'Casablanca']
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Titanic', 'Some Folks Call It a Sling Blade', 'Boot, Das', 'Braveheart', 'Cinema Paradiso']
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
['Boot, Das', 'Close Shave, A', 'Some Folks Call It a Sling Blade', 'Shall We Dance?', 'Manchurian Candidate, The']
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
['Local Hero', 'Titanic', 'Close Shave, A', 'Manchurian Candidate, The', 'Paths of Glory']
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
['Postino, Il', 'Waiting for Guffman', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Wag the Dog', 'Deconstructing Harry']
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Lawrence of Arabi

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 8s [Retry 4/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 8s [Retry 5/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
HTTP Error 429 thrown while requesting HEAD https://hugg

['Shawshank Redemption, The', 'Titanic', 'Boot, Das', 'Princess Bride, The', 'Good Will Hunting']
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
['Shawshank Redemption, The', 'Much Ado About Nothing', 'Dances with Wolves', 'Miracle on 34th Street', 'Some Folks Call It a Sling Blade']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Close Shave, A', 'Manchurian Candidate, The', "Schindler's List", 'Casablanca', "One Flew Over the Cuckoo's Nest"]
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Postino, Il', 'Boot, Das', 'Shall We Dance?', 'Close Shave, A']
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Shawshank Redemption, The', 'Boot, Das', 'Braveheart', 'Some Folks Call It a Sling Blade', 'Empire Strikes Back, The']
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Close Shave, A', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Godfather, The', '12 Angry Men', 'Citizen Kane']
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Some Folk

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 8s [Retry 4/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 8s [Retry 5/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
HTTP Error 429

['GoodFellas', 'Koyaanisqatsi', 'Farewell My Concubine', 'Shall We Dance?', 'As Good As It Gets']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Lawrence of Arabia', 'Titanic', "Singin' in the Rain", 'Good Will Hunting', 'Godfather, The']
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Shawshank Redemption, The', 'Boot, Das', 'Close Shave, A', 'Braveheart', 'Kolya']
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Lawrence of Arabia', 'Titanic', 'Close Shave, A', 'Some Folks Call It a Sling Blade', 'Manchurian Candidate, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Titanic', 'Close Shave, A', 'Boot, Das', 'Manchurian Candidate, The', 'Star Wars']
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
['Close Shave, A', 'Boot, Das', 'Thin Man, The', 'Manchurian Candidate, The', 'Godfather, The']
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Trainspotting', 'Raging Bull', 'Postino, Il', 'Clockwork Orange, A']
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Close 

...

["Schindler's List", 'Titanic', 'Empire Strikes Back, The', 'Shawshank Redemption, The', 'Wrong Trousers, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
["Schindler's List", 'Shawshank Redemption, The', 'Raiders of the Lost Ark', 'Star Wars', 'Titanic']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
["Schindler's List", 'Titanic', 'Star Wars', 'Casablanca', 'Godfather, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Star Wars', 'Titanic', 'Casablanca', "Schindler's List", 'Shawshank Redemption, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Casablanca', 'Boot, Das', "Schindler's List", 'Close Shave, A', 'Star Wars']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Kull the Conqueror', 'Raiders of the Lost Ark', 'Washington Square', 'Kull the Conqueror', 'Soul Food']
Erro ao gerar recomendações ou calcular métricas para o usuário 845: Categorical categories must be unique
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Top Gun', 'Dirty Dancing', 'Oliver & Company', 'Crimson Tide', 'Return of the Jedi']
52/52 ━━━

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/sentence_bert_config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/sentence_bert_config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/sentence_bert_config.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/sentence_bert_config.json
Retrying in 8s [Retry 4/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sent

['Pulp Fiction', 'Postino, Il', 'Henry V', 'Cook the Thief His Wife & Her Lover, The', 'Sweet Hereafter, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Raiders of the Lost Ark', "Schindler's List", 'Star Wars', 'Shawshank Redemption, The', 'Much Ado About Nothing']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Titanic', 'Third Man, The', 'Close Shave, A', 'Wallace & Gromit: The Best of Aardman Animation', 'Wrong Trousers, The']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
['Boot, Das', 'Star Wars', 'Shawshank Redemption, The', 'Usual Suspects, The', "Schindler's List"]
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Casablanca', 'Close Shave, A', 'Godfather, The', 'Wrong Trousers, The', 'Rear Window']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Star Wars', 'Godfather, The', 'Shawshank Redemption, The', 'Titanic', 'Rear Window']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Titanic', "Schindler's List", 'Shawshank Redemption, The', 'Star Wars', 'Casablanca']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Star Wars', '

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 8s [Retry 4/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
Retrying in 8s [Retry 5/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json
HTTP Error 429

["Schindler's List", 'Titanic', 'Casablanca', 'Shawshank Redemption, The', 'Raiders of the Lost Ark']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
['Casablanca', 'Wrong Trousers, The', 'North by Northwest', 'Third Man, The', '12 Angry Men']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
["Schindler's List", 'Close Shave, A', 'Shawshank Redemption, The', 'Boot, Das', 'Some Folks Call It a Sling Blade']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Third Man, The', 'Wizard of Oz, The', 'Rebel Without a Cause', 'To Kill a Mockingbird', 'Vertigo']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
["Schindler's List", 'Titanic', 'Boot, Das', 'Casablanca', 'Rear Window']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Godfather, The', 'Leaving Las Vegas', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Citizen Kane', 'Secrets & Lies']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['Good Will Hunting', "Schindler's List", 'Star Wars', 'Boot, Das', 'Apt Pupil']
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
["Schindler'